In [ ]:
import numpy as np
import utility_b as U

Αλγόριθμος RANSAC

1. Επίλεξε τυχαία τον ελάχιστον αριθμό σημείων που απαιτοούνται για τον προσδιορισμό των παραμέτρων του μοντέλου.
2. Λύσε το πρόβλημα με τις παραμέτρους του μοντέλου.
3. Προσδιόρισε πόσα σημεία από το σύνολο όλων των σημείων ταιριάζουν με μια προκαθορισμένη ανοχή.
4. Εαν το κλάσμα του πλήθος των outliers δια το πλήθος όλων των σημείων υπερβαίνει ένα προκαθορισμένο όριο t, επίλεξε νέες παραμέτρους βάση των inliers.
5. Διαφορετικά, επανέλαβε τα βήματα 1 - 4 (το πολύ Ν φορές)

inliers -> σημεία του plane που θέλουμε να βρούμε ώστε να τα αφαιρέσουμε
outlier -> σημεία που δεν ανήκουν σε κάποιο plane

In [ ]:
# load pointcloud
vertices = np.load('../pointcloud/gt_vertices.npy')
colors = np.load('../pointcloud/gt_colors.npy')

print(vertices.shape)
print(colors.shape)

In [ ]:
# ===================== GRAOUND TROUTH POINTCLOUD =====================
n_iterations = 1200
# threashold for graound trouth will be 150
# threashold for my pointcloud will be 20

threshold_distance = 150  # Define a threshold distance to consider a point as an inlier

gt_total_inlier_points = np.array([])
gt_total_inlier_colors = np.array([])
gt_total_outlier_points = np.array([])
gt_total_outlier_colors = np.array([])

# Run the RANSAC algorithm

best_plane, best_inliers = U.ransac_plane_fit(vertices, n_iterations, threshold_distance)

# Access the best fit plane coefficients
# a, b, c, d = best_plane

# Access the inlier points from the point cloud
gt_inlier_points = vertices[best_inliers]
gt_inlier_colors = colors[best_inliers]
print("inlier points shape: ", gt_inlier_points.shape)
print("inlier colors shape: ", gt_inlier_colors.shape)

# Access the outlier points from the point cloud
gt_outliers_indecies = np.delete(np.arange(vertices.shape[0]), best_inliers)
gt_outlier_points = vertices[gt_outliers_indecies]
gt_outlier_colors = colors[gt_outliers_indecies]
print("outlier points shape: ", gt_outlier_points.shape)
print("outlier colors shape: ", gt_outlier_colors.shape)

# append each point to total points
gt_total_inlier_points = gt_inlier_points
gt_total_inlier_colors = gt_inlier_colors
gt_total_outlier_points = gt_outlier_points
gt_total_outlier_colors = gt_outlier_colors

n_iterations = 1200
threshold_distance = 2000
threshold_color = 0.3
vertices = gt_outlier_points
colors = gt_outlier_colors
best_inliers = U.ransac_line_fit(vertices, colors, n_iterations, threshold_distance, threshold_color)

# Access the inlier points from the point cloud
gt_inlier_points2 = vertices[best_inliers]
gt_inlier_colors2 = colors[best_inliers]
print("inlier points shape: ", gt_inlier_points.shape)
print("inlier colors shape: ", gt_inlier_colors.shape)

# Access the outlier points from the point cloud
outliers_indecies = np.delete(np.arange(vertices.shape[0]), best_inliers)
gt_outlier_points2 = vertices[outliers_indecies]
gt_outlier_colors2 = colors[outliers_indecies]
print("outlier points shape: ", gt_outlier_points2.shape)
print("outlier colors shape: ", gt_outlier_colors2.shape)

gt_total_inlier_points = np.vstack((gt_total_inlier_points, gt_inlier_points2))
gt_total_inlier_colors = np.vstack((gt_total_inlier_colors, gt_inlier_colors2))
gt_total_outlier_points = gt_outlier_points2
gt_total_outlier_colors = gt_outlier_colors2

# save inlier points and colors
np.save('../pointcloud/gt_planes.npy', gt_total_inlier_points)
np.save('../pointcloud/gt_planes_colors.npy', gt_total_inlier_colors)

# save outlier points and colors
np.save('../pointcloud/gt_objects.npy', gt_total_outlier_points)
np.save('../pointcloud/gt_objects_colors.npy', gt_total_outlier_colors)


In [ ]:
# load my pointcloud
vertices = np.load('../pointcloud/vertices.npy')
colors = np.load('../pointcloud/colors.npy')

print(vertices.shape)
print(colors.shape)

In [ ]:
# ===================== MY POINTCLOUD =====================

# wall plane
n_iterations = 1200

threshold_distance = 35  # Define a threshold distance to consider a point as an inlier

my_total_inlier_points = np.array([])
my_total_inlier_colors = np.array([])


# Run the RANSAC algorithm
best_plane, best_inliers = U.ransac_plane_fit(vertices, n_iterations, threshold_distance)

# Access the inlier points from the point cloud
my_inlier_points = vertices[best_inliers]
my_inlier_colors = colors[best_inliers]
print("inlier points shape: ", my_inlier_points.shape)
print("inlier colors shape: ", my_inlier_colors.shape)


my_total_inlier_points = my_inlier_points
my_total_inlier_colors = my_inlier_colors


# Access the outlier points from the point cloud
outliers_indecies = np.delete(np.arange(vertices.shape[0]), best_inliers)
my_outlier_points = vertices[outliers_indecies]
my_outlier_colors = colors[outliers_indecies]
print("outlier points shape: ", my_outlier_points.shape)
print("outlier colors shape: ", my_outlier_colors.shape)


n_iterations = 50
threshold_distance = 200
color_threshold_distance = 0.6
neighbors_inlier_indicies, my_outlier_points, my_outlier_colors = U.neighbors_inlier(my_total_inlier_points,my_outlier_points,my_total_inlier_colors, my_outlier_colors, n_iterations, threshold_distance, color_threshold_distance)
print("neighbors_inlier_indicies shape: ", neighbors_inlier_indicies.shape)

# append each point to total points
neighbors_inlier = my_outlier_points[neighbors_inlier_indicies]
neighbors_inlier_colors = my_outlier_colors[neighbors_inlier_indicies]

# # trasnfom best inlier color to red 
# inlier_colors[:,0] = 1
# inlier_colors[:,1] = 0
# inlier_colors[:,2] = 0

# # transform neightbours inlier color to green
# neighbors_inlier_colors[:,0] = 0
# neighbors_inlier_colors[:,1] = 1
# neighbors_inlier_colors[:,2] = 0

total_inlier_points = np.vstack((my_total_inlier_points, neighbors_inlier))
total_inlier_colors = np.vstack((my_total_inlier_colors, neighbors_inlier_colors))

my_outlier_points = np.delete(my_outlier_points, neighbors_inlier_indicies, axis=0)
my_outlier_colors = np.delete(my_outlier_colors, neighbors_inlier_indicies, axis=0)


# road plane
# threshold_distance = 10
# n_iterations = 1200
# vertices = outlier_points
# best_plane, best_inliers = U.ransac_plane_fit(vertices, n_iterations, threshold_distance)

# # Access the inlier points from the point cloud
# inlier_points = vertices[best_inliers]
# inlier_colors = colors[best_inliers]
# print("inlier points shape: ", inlier_points.shape)
# print("inlier colors shape: ", inlier_colors.shape)

# total_inlier_points = np.vstack((total_inlier_points, inlier_points))
# total_inlier_colors = np.vstack((total_inlier_colors, inlier_colors))

# outlier_points = np.delete(outlier_points, best_inliers, axis=0)
# outlier_colors = np.delete(outlier_colors, best_inliers, axis=0)

# # save inlier points and colors
np.save('../pointcloud/my_planes.npy', my_total_inlier_points)
np.save('../pointcloud/my_planes_colors.npy', my_total_inlier_colors)

# save outlier points and colors
np.save('../pointcloud/my_objects.npy', my_outlier_points)
np.save('../pointcloud/my_objects_colors.npy', my_outlier_colors)

In [ ]:
# ===================== GROUND TRUTH POINTCLOUD =====================
# import pointcloud as pc
import numpy as np
vertices = np.load('../pointcloud/gt_objects.npy')
colors = np.load('../pointcloud/gt_objects_colors.npy')

print(vertices.shape)
print(colors.shape)

In [ ]:
# label 0 -> color green
# label 1 -> color red
# label 2 -> color blue
# label 3 -> color yellow
# label 4 -> color pink
# label 5 -> color cyan
# label 6 -> color orange
# label 7 -> color purple
# label 8 -> color brown
# label 9 -> color gray
# label 10 -> color white

color_dict = {
    0: [0, 1, 0], 
    1: [1, 0, 0], 
    2: [0, 0, 1], 
    3: [1, 1, 0], 
    4: [1, 0, 1], 
    5: [0, 1, 1], 
    6: [1, 0.5, 0],
    7: [0.5, 0, 1],
    8: [0.5, 0.25, 0],
    9: [0.5, 0.5, 0.5],
    10: [1, 1, 1]
}

In [ ]:
from sklearn.cluster import DBSCAN

# Compute 1st DBSCAN for identifying objetcs
db = DBSCAN(eps=1000, min_samples=4000).fit(vertices)
labels = db.labels_
first_clustering_vertices = vertices[labels > -1]
first_clustering_colors = colors[labels > -1]

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

print("Number of clusters:", n_clusters)
print("Total number of clustered vertices:",first_clustering_vertices.shape)

# Count the length of each cluster
unique_labels, cluster_counts = np.unique(labels, return_counts=True)

# print the length of each cluster
for label, count in zip(unique_labels, cluster_counts):
    if label != -1:  # Exclude the outlier label (-1)
        print("Cluster {}: {}".format(label, count))

# Create an array for all clustered points
unique_colors = np.zeros((first_clustering_vertices.shape[0], 3))


i = 0
for point in range(vertices.shape[0]):
    # get the label of the point
    label = labels[point]
    if label == -1:
        continue
    # get the color of the label
    color = color_dict[label]
    # assign the color to the point
    unique_colors[i] = color
    i += 1



In [ ]:
np.save('../pointcloud/gt_objects_clustered.npy', first_clustering_vertices)
np.save('../pointcloud/gt_objects_colors_clustered.npy', unique_colors)

In [ ]:
# remove the objects (first clusteriing) from the vertices and colors
remaining_vertices = vertices[labels == -1]
remaining_colors = colors[labels == -1]
print("remaining vertices after 1st clustering shape: ", remaining_vertices.shape)

In [ ]:
np.save('../pointcloud/gt_objects_clustered.npy', remaining_vertices)
np.save('../pointcloud/gt_objects_colors_clustered.npy', remaining_colors)

In [ ]:
# Compute 2nd DBSCAN for adentify objects
db = DBSCAN(eps=1000, min_samples=1000).fit(remaining_vertices)

labels = db.labels_
second_clustering_vertices = remaining_vertices[labels > -1]
second_clustering_colors = remaining_colors[labels > -1]

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

print("Number of clusters:", n_clusters)
print("Total number of clustered vertices:",second_clustering_vertices.shape)

# Count the length of each cluster
unique_labels, cluster_counts = np.unique(labels, return_counts=True)

# print the length of each cluster
for label, count in zip(unique_labels, cluster_counts):
    if label != -1:  # Exclude the outlier label (-1)
        print("Cluster {}: {}".format(label, count))

# Create an array for all clustered points
unique_colors = np.zeros((second_clustering_vertices.shape[0], 3))

i = 0
for point in range(remaining_vertices.shape[0]):
    # get the label of the point
    label = labels[point]
    if label == -1:
        continue
    # get the color of the label
    color = color_dict[label]
    # assign the color to the point
    unique_colors[i] = color
    i += 1

In [ ]:
np.save('../pointcloud/gt_objects_clustered.npy', second_clustering_vertices)
np.save('../pointcloud/gt_objects_colors_clustered.npy', second_clustering_colors)

In [ ]:
# remaining points after 3rd clustering
remaining_vertices2 = remaining_vertices[labels == -1]
remaining_colors2 = remaining_colors[labels == -1]

print("remaining vertices after 3rd clustering shape: ", remaining_vertices2.shape)

In [ ]:
np.save('../pointcloud/gt_objects_clustered.npy', remaining_vertices2)
np.save('../pointcloud/gt_objects_colors_clustered.npy', remaining_colors2)

In [ ]:
# Compute DBSCAN for adentify objects
db = DBSCAN(eps=500, min_samples=300).fit(remaining_vertices2)

labels = db.labels_
third_clustering_vertices = remaining_vertices2[labels > -1]
third_clustering_colors = remaining_colors2[labels > -1]

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

print("Number of clusters:", n_clusters)
print("Total number of clustered vertices:",third_clustering_vertices.shape)

# Count the length of each cluster
unique_labels, cluster_counts = np.unique(labels, return_counts=True)

# print the length of each cluster
for label, count in zip(unique_labels, cluster_counts):
    if label != -1:  # Exclude the outlier label (-1)
        print("Cluster {}: {}".format(label, count))

# Create an array for all clustered points
unique_colors = np.zeros((third_clustering_vertices.shape[0], 3))

i = 0
for point in range(remaining_vertices2.shape[0]):
    # get the label of the point
    label = labels[point]
    if label == -1:
        continue
    # get the color of the label
    color = color_dict[label]
    # assign the color to the point
    unique_colors[i] = color
    i += 1

In [ ]:
np.save('../pointcloud/gt_objects_clustered.npy', third_clustering_vertices)
np.save('../pointcloud/gt_objects_colors_clustered.npy', unique_colors)

In [ ]:
# add all the objexts together

gt_total_vertices = np.vstack((first_clustering_vertices, second_clustering_vertices, third_clustering_vertices))
gt_total_colors = np.vstack((first_clustering_colors, second_clustering_colors, third_clustering_colors))

print("total vertices shape: ", gt_total_vertices.shape)

In [ ]:
# add the planes to the total vertices
gt_total_vertices = np.vstack((gt_total_vertices, gt_total_inlier_points))
gt_total_colors = np.vstack((gt_total_colors, gt_total_inlier_colors))

print("total vertices shape: ", gt_total_vertices.shape)

In [ ]:
np.save('../pointcloud/gt_objects_clustered.npy', gt_total_vertices)
np.save('../pointcloud/gt_objects_colors_clustered.npy', gt_total_colors)